# Example Problem

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
n_steps=5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(size=n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))

In [ ]:
dataset = dataset.shuffle(buffer_size=10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print('_' * 20, 'Batch', index, '\nX_batch')
    print(X_batch.numpy())
    print('=' * 5, '\nY)batch')
    print(Y_batch.numpy())

In [ ]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [ ]:
print(shakespeare_text[:250])

In [ ]:
"".join(sorted(set(shakespeare_text.lower())))

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts(shakespeare_text)

In [ ]:
tokenizer.texts_to_sequences(['sample'])

In [ ]:
tokenizer.sequences_to_texts([[8, 5, 15, 23, 12, 2]])

In [ ]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print(max_id)
print(dataset_size)

In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [ ]:
dataset = dataset.prefetch(1)

In [ ]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(units=128, return_sequences=True, input_shape=[None, max_id], 
                        dropout=.2, recurrent_dropout=.2),
    tf.keras.layers.GRU(128, return_sequences=True, dropout=.2, recurrent_dropout=.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=max_id, activation=tf.nn.softmax))
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, steps_per_epoch=train_size//batch_size, epochs=10)

Problem 8

In [1]:
from datetime import date
import tensorflow as tf

MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 1, 1).toordinal()
    
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]
    
    x = [MONTHS[dt.month - 1] + ' ' + dt.strftime('%d, %Y') for dt in dates]
    y = [dt.isoformat() for dt in dates]
    
    return x, y

In [2]:
import numpy as np

np.random.seed(42)
n_dates = 3
x_example, y_example = random_dates(n_dates)
print('{:25s}{:25s}'.format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [3]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS)))) + "0123456789, "
INPUT_CHARS

'ADFJMNOSabceghilmnoprstuvy0123456789, '

In [4]:
OUTPUT_CHARS = "0123456789-"

In [5]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [6]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X+1).to_tensor() # Using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)


In [7]:
np.random.seed(42)

X_train, y_train = create_dataset(10000)
X_valid, y_valid = create_dataset(2000)
X_test, y_test = create_dataset(2000)

In [8]:
y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

In [14]:
embedding_size = 32
max_output_length = y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=len(INPUT_CHARS)+1,
                              output_dim=embedding_size,
                              input_shape=[None]),
    tf.keras.layers.LSTM(128)
])

decoder = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(units=len(OUTPUT_CHARS)+1, activation=tf.nn.softmax)
])

model = tf.keras.models.Sequential([encoder, 
                                   tf.keras.layers.RepeatVector(max_output_length),
                                   decoder])
optimizer = tf.keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid,y_valid))

Train on 10000 samples, validate on 2000 samples
Epoch 1/20
10000/10000 [==============================] - 16s 2ms/sample - loss: 1.8177 - accuracy: 0.3482 - val_loss: 1.4038 - val_accuracy: 0.4696
Epoch 2/20
10000/10000 [==============================] - 11s 1ms/sample - loss: 1.3489 - accuracy: 0.5124 - val_loss: 1.2919 - val_accuracy: 0.5418
Epoch 3/20
10000/10000 [==============================] - 13s 1ms/sample - loss: 1.0756 - accuracy: 0.6150 - val_loss: 0.9297 - val_accuracy: 0.6532
Epoch 4/20
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.8656 - accuracy: 0.6827 - val_loss: 0.7471 - val_accuracy: 0.7216
Epoch 5/20
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.6145 - accuracy: 0.7685 - val_loss: 0.5148 - val_accuracy: 0.8015
Epoch 6/20
10000/10000 [==============================] - 12s 1ms/sample - loss: 0.5421 - accuracy: 0.8042 - val_loss: 0.5393 - val_accuracy: 0.8061
Epoch 7/20
10000/10000 [==============================] -

# Problem 11